# With this notebook, we will be trying to gain a better understanding of the Purple Air API.

In [2]:
#load in libraries
import requests
import json
import pandas as pd


### Testing Api connection

In [3]:
#authentication keys
Read_Key = "C7AF49F7-C3F6-11ED-B6F4-42010A800007"
Write_Key = "C7B07520-C3F6-11ED-B6F4-42010A800007"

In [4]:
# Establish initial connection 

# Set up the API endpoint URL and headers
url = "https://api.purpleair.com/v1/keys"
headers = {"X-API-Key": Read_Key}

# Make the API request
response = requests.get(url, headers=headers)

# Print the API response
print(response.json())


{'api_version': 'V1.0.11-0.0.44', 'time_stamp': 1683721300, 'api_key_type': 'READ'}


#### Connection was successful 

### Next, Test pulling sensor data

How to make a call : https://community.purpleair.com/t/making-api-calls-with-the-purpleair-api/180

How to find sensor index : https://community.purpleair.com/t/sensor-index/4000

In [5]:
#Sensor ID for testing
Union_Bethel_AME = "133730#11.74"
William_S_Schmidt = "134488#11.74"

sensor_id_dict = {"Union_Bethel_AME": "133730#11.74", 
                  "William_S_Schmidt": "134488#11.74",
                  "PGCPS_Schmidt_CenterBldg": "102898#11",
                  "Surratsville_HS": "102852#11.05",
                  "Oxon_Hill_HS": "104790#11.05"} # Replace with your own sensor IDs and locations


In [6]:
#new url for sensor data
endpoint = "https://api.purpleair.com/v1/sensors/"

# Define API headers with your X-API-Key
headers = {
    "X-API-Key": Read_Key
}

# Make API GET request
response = requests.get(endpoint + Union_Bethel_AME, headers=headers)

# Check if request was successful
if response.status_code == 200:
    # Get data from response
    data = response.json()
    # Print data
    print(data)
else:
    # Print error message
    print("Error:", response.status_code)
    
response.text

{'api_version': 'V1.0.11-0.0.44', 'time_stamp': 1683721305, 'data_time_stamp': 1683721253, 'sensor': {'sensor_index': 133730, 'last_modified': 1676581473, 'date_created': 1634686448, 'last_seen': 1683721135, 'private': 0, 'is_owner': 0, 'name': 'Union Bethel AME', 'icon': 0, 'location_type': 0, 'model': 'PA-II-SD', 'hardware': '2.0+OPENLOG+NO-DISK+DS3231+BME280+PMSX003-B+PMSX003-A', 'led_brightness': 50, 'firmware_version': '7.02', 'rssi': -71, 'uptime': 46209, 'pa_latency': 203, 'memory': 15896, 'position_rating': 5, 'latitude': 38.70654, 'longitude': -76.8886, 'altitude': 236, 'channel_state': 3, 'channel_flags': 0, 'channel_flags_manual': 0, 'channel_flags_auto': 0, 'confidence': 100, 'confidence_auto': 100, 'confidence_manual': 100, 'humidity': 52, 'humidity_a': 52, 'temperature': 63, 'temperature_a': 63, 'pressure': 1014.57, 'pressure_a': 1014.57, 'analog_input': 0.0, 'pm1.0': 0.8, 'pm1.0_a': 1.4, 'pm1.0_b': 0.2, 'pm2.5': 3.3, 'pm2.5_a': 3.7, 'pm2.5_b': 2.9, 'pm2.5_alt': 2.2, 'pm2

'{\n  "api_version" : "V1.0.11-0.0.44",\n  "time_stamp" : 1683721305,\n  "data_time_stamp" : 1683721253,\n  "sensor" : {\n    "sensor_index" : 133730,\n    "last_modified" : 1676581473,\n    "date_created" : 1634686448,\n    "last_seen" : 1683721135,\n    "private" : 0,\n    "is_owner" : 0,\n    "name" : "Union Bethel AME",\n    "icon" : 0,\n    "location_type" : 0,\n    "model" : "PA-II-SD",\n    "hardware" : "2.0+OPENLOG+NO-DISK+DS3231+BME280+PMSX003-B+PMSX003-A",\n    "led_brightness" : 50,\n    "firmware_version" : "7.02",\n    "rssi" : -71,\n    "uptime" : 46209,\n    "pa_latency" : 203,\n    "memory" : 15896,\n    "position_rating" : 5,\n    "latitude" : 38.70654,\n    "longitude" : -76.8886,\n    "altitude" : 236,\n    "channel_state" : 3,\n    "channel_flags" : 0,\n    "channel_flags_manual" : 0,\n    "channel_flags_auto" : 0,\n    "confidence" : 100,\n    "confidence_auto" : 100,\n    "confidence_manual" : 100,\n    "humidity" : 52,\n    "humidity_a" : 52,\n    "temperature" :

#### Above is a raw output of the current metrics a sensor has. This seems problematic becuase no one on our team has knowledge on Air quality terms.

#### We will learn these terms to then pull insihgts or at the very least be able to create a table for the data that we can pull. 

#### Next thing to do is attempt to pull multiple sensors at once, 
#### and pull historical data. https://api.purpleair.com/#api-sensors-get-sensor-history

In [7]:

def Histrocial_call_link (sensor_ids, start="1weekago", end="now", interval="hourly"):
    if isinstance(sensor_ids, str):
        sensor_ids = [sensor_ids]
    data_dict = {}
    for sensor_id in sensor_ids:
        url = f"https://api.purpleair.com/v1/sensors/{sensor_id}/stats?from={start}&to={end}&interval={interval}"
        response = requests.get(url, headers=headers)
        data_dict[sensor_id] = response.json()
    return data_dict

In [8]:
test = Histrocial_call_link(sensor_id_dict["Union_Bethel_AME"])
print(test)

{'133730#11.74': {'api_version': 'V1.0.11-0.0.44', 'time_stamp': 1683721310, 'data_time_stamp': 1683721261, 'sensor': {'sensor_index': 133730, 'last_modified': 1676581473, 'date_created': 1634686448, 'last_seen': 1683721254, 'private': 0, 'is_owner': 0, 'name': 'Union Bethel AME', 'icon': 0, 'location_type': 0, 'model': 'PA-II-SD', 'hardware': '2.0+OPENLOG+NO-DISK+DS3231+BME280+PMSX003-B+PMSX003-A', 'led_brightness': 50, 'firmware_version': '7.02', 'rssi': -70, 'uptime': 46211, 'pa_latency': 195, 'memory': 15896, 'position_rating': 5, 'latitude': 38.70654, 'longitude': -76.8886, 'altitude': 236, 'channel_state': 3, 'channel_flags': 0, 'channel_flags_manual': 0, 'channel_flags_auto': 0, 'confidence': 100, 'confidence_auto': 100, 'confidence_manual': 100, 'humidity': 51, 'humidity_a': 51, 'temperature': 63, 'temperature_a': 63, 'pressure': 1014.54, 'pressure_a': 1014.54, 'analog_input': 0.0, 'pm1.0': 0.6, 'pm1.0_a': 1.0, 'pm1.0_b': 0.1, 'pm2.5': 4.0, 'pm2.5_a': 4.4, 'pm2.5_b': 3.6, 'pm2.